### Template tissue creator
This notebook is designed to create tissue map templates from the segmented infant data. 

In [ ]:
#import packages
from os import listdir
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber # Data i/o
from nipype.interfaces.utility import IdentityInterface, Function     # utility
from nipype.pipeline.engine import Node, Workflow, MapNode
from nipype.interfaces.fsl.preprocess import FLIRT
from nipype.interfaces.fsl.maths import MeanImage
from nipype.interfaces.fsl.utils import Merge

#set output file type for FSL to NIFTI
from nipype.interfaces.fsl.preprocess import FSLCommand
FSLCommand.set_default_output_type('NIFTI')

# Set study variables
studyhome = '/Users/catcamacho/Box/SNAP/BABIES'
#studyhome = '/share/iang/active/BABIES/BABIES_rest'
raw_data = '/Users/catcamacho/Box/SNAP/BABIES/seg_temp/BABIES_C'
output_dir = studyhome + '/processed/template'
workflow_dir = studyhome + '/workflows'
subjects_list = open(studyhome + '/misc/templatesubjects.txt').read().splitlines()

template_brain = studyhome + '/templates/T2w_BABIES_template_2mm.nii'

proc_cores = 2 # number of cores of processing for the workflows

tissue_list=['wm','gm']

In [ ]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subj']),
                     name="infosource")
infosource.iterables = ('subj', subjects_list)

# Data grabber- select gm/wm
templates = {'struct': raw_data + '/T1{subj}/T1{subj}-5/skullstripped_anat.nii',
             'seg': raw_data + '/T1{subj}/T1{subj}-5/seg.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# Datasink- where our select outputs will go
substitutions = [('_subj_', '')]
datasink = Node(DataSink(), name='datasink')
datasink.inputs.base_directory = output_dir
datasink.inputs.container = output_dir
datasink.inputs.substitutions = substitutions

In [ ]:
def split_tissues(in_tissues):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from nibabel import load, save, Nifti1Image
    
    segmentation_nifti = load(in_tissues)
    segmentation_data = segmentation_nifti.get_data()

    tissue_data = segmentation_data
    tissue_data[tissue_data==150] = 1
    tissue_data[tissue_data==10] = 0
    tissue_data[tissue_data==250] = 0
    tissue_nifti = Nifti1Image(tissue_data, segmentation_nifti.affine)
    save(tissue_nifti, 'gm.nii')
    tissue_file = abspath('gm.nii')
    
    out_tissues=[tissue_file]
    
    return(out_tissues)

split_tissues = Node(Function(input_names=['in_tissues'], 
                              output_names=['out_tissues'], 
                              function=split_tissues), 
                     name='split_tissues')

register_anat = Node(FLIRT(reference=template_brain, 
                           out_matrix_file='xform.mat'), 
                     name='register_anat')

register_seg = MapNode(FLIRT(reference=template_brain,
                             apply_xfm=True),
                       name='register_seg', 
                       iterfield=['in_file'])



In [ ]:
templatetissue = Workflow(name='templatetissue')
templatetissue.connect([(infosource, selectfiles,[('subj','subj')]),
                        (selectfiles, register_anat,[('struct','in_file')]),
                        (selectfiles, split_tissues,[('seg','in_tissues')]),
                        (register_anat, register_seg, [('out_matrix_file','in_matrix_file')]),
                        (split_tissues, register_seg, [('out_tissues','in_file')]),
                        (register_anat, datasink, [('out_file','reg_anat')]),
                        (register_seg, datasink, [('out_file','reg_gm')])
                       ])
templatetissue.base_dir = workflow_dir
templatetissue.write_graph(graph2use='flat')
templatetissue.run('MultiProc', plugin_args={'n_procs': proc_cores})

In [ ]:
tissueinfo = Node(IdentityInterface(fields=['tissue']),
                     name='tissueinfo')
tissueinfo.iterables = ('tissue', tissue_list)

select_tissue = Node(DataGrabber(infields=['tissue'], 
                                 base_directory=output_dir, 
                                 sort_filelist=True,
                                 template=output_dir+'/*/*/*/%s*.nii',
                                 field_template={'tissue_data':output_dir +'/*/*/*/%s*.nii'},
                                 template_args = dict(tissue_data=[['tissue']])),
                     name='select_tissue')

In [ ]:
merge_image = Node(Merge(dimension='t'), 
                   name='merge_image')

average_image = Node(MeanImage(), 
                     name='average_image')

In [ ]:
group_tissue = Workflow(name='group_tissue')
group_tissue.connect([(tissueinfo, select_tissue, [('tissue','tissue')]),
                      (select_tissue, merge_image, [('tissue_data','in_files')]),
                      (merge_image, average_image, [('merged_file','in_file')]),
                      (average_image, datasink, [('out_file','average_tissue')])
                     ])
group_tissue.base_dir = workflow_dir
group_tissue.write_graph(graph2use='flat')
group_tissue.run('MultiProc', plugin_args={'n_procs': proc_cores})